In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
bootstraps = [str(i) for i in range(0,10)]

# plots mean AUC +/- std for each model
# Inputs: model names as a list, Pandas DataFrame of exported results CSV from Google Spreadsheets


def plot_performance(
    model_names: list,
    df: pd.DataFrame,
    fpath_bar_plot: str,
    y_min: float = 0.7,
    y_max: float = 0.95):
    y_delta = 0.05
    
    stats = {model: {'aucs': [], 'mean': 0, 'std': 0} for model in model_names}

    for model in model_names:
        stats[model]['aucs'] = list(float(auc) for auc in df.loc[bootstraps, model].values)
        stats[model]['mean'] = float(df.loc['mean', model])
        stats[model]['std'] = float(df.loc['std', model])
        
    means = [stats[model]['mean'] for model in model_names]
    stds = [stats[model]['std'] for model in model_names]
    
    sns.set_style("whitegrid")
    sns.set_context("poster")
    sns.set_palette("pastel")
    
    fig_width = len(model_names) * 3
    fig, ax = plt.subplots(figsize=(fig_width, 8))
    x_pos = np.arange(len(model_names))
    
    bar = sns.barplot(x=x_pos, y=means, yerr=stds)
    ax.set_ylabel('AUC')
    ax.set_ylim([y_min, y_max])
    ax.set_yticks(np.arange(y_min, y_max+0.01, y_delta))
    
    ax.set_xticks(range(len(model_names)))
    ax.set_xticklabels(model_names)
    ax.set_title('Model Performance')

    offset = -0.02 * len(model_names)
    for i, v in enumerate(means):
        plt.text(x_pos[i]+offset, y_min + 0.01, f'{v:0.3f}')

    plt.xticks(rotation=90)
    plt.tight_layout()
    
    plt.savefig(fpath_bar_plot, dpi=100)
    print(f"Saved {fpath}")
    
    plt.show()

In [ ]:
rootdir = os.path.expanduser("~/dropbox/sts-ecg/figures")
fpath = os.path.join(rootdir, "STS ECG models and results - architecture_and_results_3.csv")

# Load results CSV into dataframe
df = pd.read_csv(fpath, index_col=0)
df

In [ ]:
models_to_plot = []
for i in range(1,7):
    models_to_plot.append(f"v{i}")
fpath_bar_plot = os.path.join(rootdir, "aucs1.png")
plot_performance(
    model_names=models_to_plot,
    df=df,
    fpath_bar_plot=fpath_bar_plot,
    y_min=0.65,
    y_max=0.8,
)

In [ ]:
models_to_plot = []
for i in range(7,13):
    models_to_plot.append(f"v{i}")
fpath_bar_plot = os.path.join(rootdir, "aucs2.png")
plot_performance(
    model_names=models_to_plot,
    df=df,
    fpath_bar_plot=fpath_bar_plot,
    y_min=0.65,
    y_max= 0.80,
)

In [ ]:
models_to_plot = []
for i in range(13,20):
    models_to_plot.append(f"v{i}")
fpath_bar_plot = os.path.join(rootdir, "aucs3.png")
plot_performance(
    model_names=models_to_plot,
    df=df,
    fpath_bar_plot=fpath_bar_plot,
    y_min=0.65,
    y_max= 0.80,
)

In [ ]:
fpath = os.path.join(rootdir, "STS ECG models and results - architecture_and_results_4.csv")

# Load results CSV into dataframe
df = pd.read_csv(fpath, index_col=0)

print(f"Loaded dataframe from {fpath} with keys:\n")
for key in df.keys():
    print(f"\t{key}")

In [ ]:
def get_aucs_from_column(df: pd.DataFrame, col_name: str, bootstraps: int = 10) -> list:
    """Given a dataframe and model name (col_name),
    extracts the AUCs and casts to list of floats"""
    aucs = []
    if col_name not in df:
        return aucs
    for bootstrap in range(bootstraps):
        try:
            auc = df[col_name].loc[str(bootstrap)]
            aucs.append(float(auc))
        except:
            aucs.append(np.nan)
            print(f"no valid auc found at bootstrap {bootstrap}")
    return aucs

In [ ]:
# Get AUCs from dataframe as a dict (keyed by model name) of lists of floats
aucs = {}
for model in df.keys():
    aucs[model] = get_aucs_from_column(df=df, col_name=model, bootstraps=10)

In [ ]:
model_1 = 'v14'
model_2 = 'v14-new-bootstraps'

models_to_plot = [model_1, model_2]
fpath_bar_plot = os.path.join(rootdir, "aucs-compare-v14.png")
plot_performance(
    model_names=models_to_plot,
    df=df,
    fpath_bar_plot=fpath_bar_plot,
    y_min=0.65,
    y_max= 0.80,
)

statistic, p_val = stats.ttest_rel(aucs[model_1], aucs[model_2])
print(f"mean AUC = {np.mean(aucs[model_1]):0.3f} ± {np.std(aucs[model_1]):0.3f} from model {model_1} ")
print(f"mean AUC = {np.mean(aucs[model_2]):0.3f} ± {np.std(aucs[model_2]):0.3f} from model {model_2} ")
print(f"p-value = {p_val:0.3f}")

In [ ]:
model_1 = 'shallow-sts-preop'
model_2 = 'deep-sts-preop-v8'

models_to_plot = [model_1, model_2]
fpath_bar_plot = os.path.join(rootdir, "aucs-compare-preop.png")
plot_performance(
    model_names=models_to_plot,
    df=df,
    fpath_bar_plot=fpath_bar_plot,
    y_min=0.75,
    y_max= 0.95,
)

statistic, p_val = stats.ttest_rel(aucs[model_1], aucs[model_2])
print(f"mean AUC = {np.mean(aucs[model_1]):0.3f} ± {np.std(aucs[model_1]):0.3f} from model {model_1} ")
print(f"mean AUC = {np.mean(aucs[model_2]):0.3f} ± {np.std(aucs[model_2]):0.3f} from model {model_2} ")
print(f"p-value = {p_val:0.3f}")